This notebook contains some code to show how to run the Python files containing the model code.

In [ ]:
import math, os

from Code.Data import *
from Code.Load import *
from Code.CP import *
from Code.QualityMeasures import *

import numpy as np
from tqdm.notebook import tqdm

In [ ]:
data = "turbine"

In [ ]:
def summary(data, model, CP = True, folder = FOLDER):
    folder = FOLDER + "/" + data + "/results/"
    files = os.listdir(folder)
    cp = "-CP-" if CP else ""
    filtered = [file for file in files if (model+cp in file)]
    
    data = np.zeros((len(filtered), 3))
    
    for i, f in enumerate(filtered):
        with open(folder + f, "r") as file:
            parts = file.read().splitlines()[0].split("\t")
            data[i, 0] = float(parts[0])
            data[i, 1] = float(parts[1])
            data[i, 2] = float(parts[2])
            
    return {"cov": np.mean(data[:, 0]), "width": np.mean(data[:, 1]), "r2": np.mean(data[:, 2]), \
           "cov_std": np.std(data[:, 0]), "width_std": np.std(data[:, 1]), "r2_std": np.std(data[:, 2])}

In [ ]:
import Code.RunNNCP as NN

for s in tqdm(SEEDS):
    NN.run(data, s)
    
summary(data, "NN")

In [ ]:
import Code.RunRFCP as RF

for s in tqdm(SEEDS):
    RF.run(data, s)
    
summary(data, "RF")

In [ ]:
import Code.RunDropout as Drop

for s in tqdm(SEEDS):
    Drop.run(data, s, num = 50, drop = np.arange(0.05, 0.55, 0.05), l = 0)
    
summary(data, "Dropout")

In [ ]:
import Code.RunKG as KG

for s in tqdm(SEEDS):
    KG.run(data, s, num = 50, drop = np.arange(0.05, 0.55, 0.05), l = 0)
    
summary(data, "KG")

In [ ]:
import Code.RunDE as DE

for s in tqdm(SEEDS):
    DE.run(data, s, num = 5, drop = 0)
    
summary(data, "DE")

In [ ]:
import Code.RunQR as QR

for s in tqdm(SEEDS):
    QR.run(data, s)
    
summary(data, "QR")

In [ ]:
import Code.RunGP as GP

for s in tqdm(SEEDS):
    GP.run(data, s, learning_rate = 0.1, epochs = 50)

summary(data, "GP")

In [ ]:
import Code.RunSVGP as AGP

for s in tqdm(SEEDS):
    AGP.run(data, s, learning_rate = 0.01, epochs = 50)
    
summary(data, "AGP")